## Notebook that will evaluate the KNN model

In [65]:
#importing libraires to use
import sklearn.preprocessing as skl_pre
import sklearn.neighbors as skl_nb
import sklearn.model_selection as skl_ms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

#need to get the processing python file in another directory
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from process_data import process_data

In [2]:
#getting the percentage splits of our training and testing data
split_prec = {
    'train': 0.8, 
    'test': 0.2,
}
#whatever scaler we will use
scaler = skl_pre.StandardScaler()

#get the training and testing data
X_train, X_test, Y_train, Y_test = process_data(split_prec, scaler,is_random=False)
print(X_train)
# print(X_test)
print(Y_train)
# print(Y_test)

Dropped columns: ['snow']
New columns: ['good_weather' 'is_day' 'temp_fahrenheit']
Split: "train" 	[Size: 1280] 	[Prec: 0.8]
	X: (1280, 17)
	Y: (1280,)
Split: "test" 	[Size: 320] 	[Prec: 0.2]
	X: (320, 17)
	Y: (320,)
      hour_of_day  day_of_week     month   holiday   weekday  summertime  \
0        0.558988    -1.017343 -0.709870 -0.184189  0.643712    0.751527   
1       -1.625091    -1.516270  0.152211 -0.184189  0.643712    0.751527   
2        1.141408    -0.518416  1.014292 -0.184189  0.643712    0.751527   
3       -1.625091     1.477292 -0.135149 -0.184189 -1.553491    0.751527   
4        1.287014    -1.017343  0.726932 -0.184189  0.643712    0.751527   
...           ...          ...       ...       ...       ...         ...   
1275     1.723829     0.978365  0.439572 -0.184189 -1.553491    0.751527   
1276    -1.625091     0.978365  1.589013 -0.184189 -1.553491   -1.330625   
1277    -0.605854    -0.518416  0.726932 -0.184189  0.643712    0.751527   
1278    -0.023433    -0

### Training we will only be working with X_train from here on down
Now that we have our training and test data we will run a KNN model on the training data with k-fold validation for hyperparameter tunning (what is a good value of k, distance measure to use, ect...). Then once we have a decent value of k we will "retrain" the model on the entire training data set and use that model on the test data which has never been seen and use the error from that as an estimation of $E_{new}$

In [ ]:
#returns a trained KNN classifier that uses cross validation and grid search to find the best hyperparameters
def get_fitted_KNN(X,Y):
    #these are the hyperparameters that we are using Gridsearch to find
    parameters = {"n_neighbors": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], "p" : [1,2,3]}
    #running grid search over a KNN model with 25 fold cross validation.
    classifier = skl_ms.GridSearchCV(skl_nb.KNeighborsClassifier(), parameters, cv=25)
    #fit the best classifier
    classifier.fit(X,Y)
    #return it
    return classifier


Below we can see the expected $E_{train}$ of our model when we run cross validation over training to evaluate

In [86]:
# print(np.array(Y_train.values.tolist()).reshape(-1,))
for i in range(10):
    classifier = get_fitted_KNN(X_train,np.array(Y_train.values.tolist()).reshape(-1,))
    print(classifier.best_params_)
    print(classifier.best_score_)

{'n_neighbors': 11, 'p': 1}
0.8694720965309201
{'n_neighbors': 11, 'p': 1}
0.8694720965309201
{'n_neighbors': 11, 'p': 1}
0.8694720965309201
{'n_neighbors': 11, 'p': 1}
0.8694720965309201
{'n_neighbors': 11, 'p': 1}
0.8694720965309201
{'n_neighbors': 11, 'p': 1}
0.8694720965309201


KeyboardInterrupt: 

Now that we have a way to evaluate a model with cross validation we can start hyperparameter tuning. Note we are focusing on Accuracy here because that is the test statistic we care about.

In [ ]:
#this will go through several different hyperparameters to hopefully tune the KNN model and find a good one
def find_a_good_KNN(df = X_train.join(Y_train)):
    parameters = {"k":}
    x_cols = df.columns.to_list().remove("increase_stock")


In [ ]:
#unnecessary code

#this will run k_folds cross validation with the passed hyperparameters
#returns the approximation of the accuracy of the model
# def run_k_folds(df = X_train.join(Y_train), folds = 5):
#     #setting up our kfolds cross validation
#     kf = skl_ms.KFold(folds, shuffle= True)
#     #getting the column list so we can separate into X and Y
#     cols = df.columns.tolist()
#     ycol = "increase_stock"
#     #only the x columns
#     cols.remove(ycol)
#     #will be the mean accuracy of the model. This is what we return
#     score = 0
#     parameters = []
#     # print(df)
#     # print(cols)
#     #loop through all the training and test data
#     for i, (train_index, test_index) in enumerate(kf.split(df)):
#         #get the specific training data
#         train = df.loc[train_index]
#         # print(train)
#         #get the specific testing data
#         test = df.loc[test_index]
#         #make a KNN classifier and fit it 
#         classifier = get_fitted_KNN(X = train[cols], Y=train[ycol])
#         #get the score using the k folds test data set
#         print(classifier.best_params_)
#         parameters.append(classifier.best_params_)
#         score += classifier.score(test[cols], test[ycol])
#     print(parameters)
#     print(Counter(parameters))
#     return score/folds
